In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from PIL import Image
import timm
from multiprocessing import Pool
from tqdm import tqdm
from torchvision import transforms
from torch.optim.lr_scheduler import LambdaLR

In [2]:
## for removing file if required
import shutil

try:
    shutil.rmtree("/kaggle/working/train")
    
    model_file_to_delete ="/kaggle/working/models"

    if os.path.isfile(model_file_to_delete):
        os.remove(model_file_to_delete)
    
except:
    print("No such directories")

No such directories


### Loading Data

In [3]:
def load_data(data_dir):
    train_csv = os.path.join(data_dir, 'train.csv')
    test_csv = os.path.join(data_dir, 'test.csv')
    
    train = pd.read_csv(train_csv)
    test = pd.read_csv(test_csv)
    
    train_dir = os.path.join(data_dir, 'train_images/')
    test_dir = os.path.join(data_dir, 'test_images/')
    
    train['file_path'] = train['id_code'].map(lambda x: os.path.join(train_dir, '{}.png'.format(x)))
    test['file_path'] = test['id_code'].map(lambda x: os.path.join(test_dir, '{}.png'.format(x)))
    
    train['file_name'] = train["id_code"] + ".png"
    test['file_name'] = test["id_code"] + ".png"
    
    train['diagnosis'] = train['diagnosis'].astype(str)
    
    return train, test

In [4]:
data_dir = '/kaggle/input/aptos2019-blindness-detection/'
train_df, test_df = load_data(data_dir)

In [5]:
print(train_df.shape)
train_df.head()

(3662, 4)


,id_code,diagnosis,file_path,file_name
0,000c1434d8d7,2,/kaggle/input/aptos2019-blindness-detection/tr...,000c1434d8d7.png
1,001639a390f0,4,/kaggle/input/aptos2019-blindness-detection/tr...,001639a390f0.png
2,0024cdab0c1e,1,/kaggle/input/aptos2019-blindness-detection/tr...,0024cdab0c1e.png
3,002c21358ce6,0,/kaggle/input/aptos2019-blindness-detection/tr...,002c21358ce6.png
4,005b95c28852,0,/kaggle/input/aptos2019-blindness-detection/tr...,005b95c28852.png


In [6]:
print(test_df.shape)
test_df.head()

(1928, 3)


,id_code,file_path,file_name
0,0005cfc8afb6,/kaggle/input/aptos2019-blindness-detection/te...,0005cfc8afb6.png
1,003f0afdcd15,/kaggle/input/aptos2019-blindness-detection/te...,003f0afdcd15.png
2,006efc72b638,/kaggle/input/aptos2019-blindness-detection/te...,006efc72b638.png
3,00836aaacf06,/kaggle/input/aptos2019-blindness-detection/te...,00836aaacf06.png
4,009245722fa4,/kaggle/input/aptos2019-blindness-detection/te...,009245722fa4.png


### Pre-Processing

In [7]:
def crop_img(img, percentage):
    
    img_arr = np.array(img)
    img_gray = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
    
    threshold = img_gray > 0.1 * np.mean(img_gray[img_gray != 0])
    row_sums = np.sum(threshold, axis=1)
    col_sums = np.sum(threshold, axis=0)
    
    rows = np.where(row_sums > img_arr.shape[1] * percentage)[0]
    cols = np.where(col_sums > img_arr.shape[0] * percentage)[0]
    
    min_row, min_col = np.min(rows), np.min(cols)
    max_row, max_col = np.max(rows), np.max(cols)
    
    crop_img = img_arr[min_row : max_row + 1, min_col : max_col + 1]
    
    return Image.fromarray(crop_img)

In [8]:
def resize_maintain_aspect(img, desired_size):
    old_width, old_height = img.size
    aspect_ratio = old_width / old_height

    if aspect_ratio > 1:
        new_width = desired_size
        new_height = int(desired_size / aspect_ratio)
    else:
        new_height = desired_size
        new_width = int(desired_size * aspect_ratio)

    resized_img = img.resize((new_width, new_height), Image.ANTIALIAS)
    
    padded_image = Image.new("RGB", (desired_size, desired_size))
    x_offset = (desired_size - new_width) // 2
    y_offset = (desired_size - new_height) // 2
    padded_image.paste(resized_img, (x_offset, y_offset))
    
    return padded_image

In [9]:
def save_single(args):
    image_path, output_path_folder, percentage, output_size = args
    image = Image.open(image_path)
    
    # Display the image
    #plt.imshow(image)
    #plt.title('Original Image')
    #plt.show()
    
    croped_img = crop_img(image,percentage)
    image_resized = resize_maintain_aspect(croped_img, desired_size=output_size[0])
    
    #print(output_path_folder)
    #print(image_path)
    output_image_path = os.path.basename(image_path)
    # Save the resized image
    output_file_path = os.path.join(output_path_folder, output_image_path)
    #print(output_file_path)
    image_resized.save(output_file_path)

In [10]:
def fast_image_resize(df, output_path_folder, percentage, output_size=None):
    """Uses multiprocessing to make it fast"""
    if not output_size:
        warnings.warn("Need to specify output_size! For example: output_size=100")
        return

    if not os.path.exists(output_path_folder):
        os.makedirs(output_path_folder)
        
    jobs = []
    for df_item in range(len(df)):
        image_path = df.file_path.iloc[df_item]
        #print(image_path)
        job = (image_path, output_path_folder, percentage, output_size)
        jobs.append(job)
    
    """
    results = []
    for job in tqdm(jobs, total=len(jobs)):
        result = save_single(job)
        results.append(result)
    """
    with Pool() as p:
        list(tqdm(p.imap_unordered(save_single, jobs), total=len(jobs)))

In [11]:
percentage = 0.01
fast_image_resize(train_df, "/kaggle/working/train/images_resized_150/",percentage, output_size=(100, 100))

  0%|          | 0/3662 [00:00<?, ?it/s]/tmp/ipykernel_34/3153832717.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_img = img.resize((new_width, new_height), Image.ANTIALIAS)
  0%|          | 1/3662 [00:00<13:33,  4.50it/s]/tmp/ipykernel_34/3153832717.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_img = img.resize((new_width, new_height), Image.ANTIALIAS)
  0%|          | 2/3662 [00:00<13:56,  4.38it/s]/tmp/ipykernel_34/3153832717.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_img = img.resize((new_width, new_height), Image.ANTIALIAS)
/tmp/ipykernel_34/3153832717.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resam

### Model Implementation - Ensambling 
#### EfficientNet b0, b1, b2, b3

In [33]:
class BlindnessDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, augmentations=None, max_count=None, test=False):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        #self.augmentations = augmentations
        self.max_count = max_count
        self.test = test
        
        if not test:
            self.class_counts = self.annotations['diagnosis'].value_counts().sort_index()
        else:
            self.class_counts = None
        
        if max_count:
            self.oversample(max_count)
    
    def oversample(self, max_count): ## Over sampling classes to balance
        samples = []
        for diagnosis in self.class_counts.index:
            class_samples = self.annotations[self.annotations['diagnosis'] == diagnosis]
            oversampled_class = class_samples.sample(max_count, replace=True)
            samples.append(oversampled_class)
        self.annotations = pd.concat(samples).reset_index(drop=True)
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0] + '.png')
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.test:
            return image
                
        label = int(self.annotations.iloc[idx, 1])
        return image, label

In [34]:
def select_model(model_name):
    if model_name == 'efficientnet_b0':
        return timm.create_model('efficientnet_b0', pretrained=True, num_classes=5)
    elif model_name == 'efficientnet_b1':
        return timm.create_model('efficientnet_b1', pretrained=True, num_classes=5)
    elif model_name == 'efficientnet_b2':
        return timm.create_model('efficientnet_b2', pretrained=True, num_classes=5)
    elif model_name == 'efficientnet_b3':
        return timm.create_model('efficientnet_b3', pretrained=True, num_classes=5)

In [35]:
def lr_schedule(epoch):
    if epoch < 10:
        return 5e-4
    elif epoch < 16:
        return 1e-4
    elif epoch < 22:
        return 1e-5
    else:
        return 1e-3
        

In [36]:
def train_model(model_name,train_loader,valid_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = select_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    
    def lambda_epoch(epoch):
        return lr_schedule(epoch)
    
    def quadratic_weighted_kappa(y_true, y_pred):
        return cohen_kappa_score(y_true, y_pred, weights='quadratic')
    
    scheduler = LambdaLR(optimizer,lr_lambda = lambda_epoch)
    best_kappa = 0.0
    
    for epoch in range(25):
        model.train()
        for images, labels in train_loader:
            if isinstance(images, Image.Image):  # Check if the image is a PIL image
                transform = transforms.ToTensor()
                images = transform(images)
            else:
                images = images
                
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        scheduler.step()
        
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels_epoch = []
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs,labels)
                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                val_labels_epoch.extend(labels.cpu().numpy())
                val_preds.extend(preds.cpu().numpy())
        
        #val_predictions.append(val_preds)
        #val_labels.append(val_labels_epoch)
        
        kappa = quadratic_weighted_kappa(val_labels_epoch, val_preds)
        print(f"Epoch {epoch+1}/{25}, Validation Loss: {val_loss / len(valid_loader)}")
        print(f"Validation QWK: {kappa}")
        
        if kappa > best_kappa:
            best_kappa = kappa
            best_model = model.state_dict()
            
    model.load_state_dict(best_model)
    
    return model

    

In [37]:
csv_file = '/kaggle/input/aptos2019-blindness-detection/train.csv'
pros_img_dir = '/kaggle/working/train/images_resized_150'
test_csv_file = '/kaggle/input/aptos2019-blindness-detection/test.csv'
test_root_dir = '/kaggle/input/aptos2019-blindness-detection/test_images'
val_csv_file = '/kaggle/working/val.csv'
val_root_dir = '/kaggle/working/val_images/'

os.makedirs(val_root_dir, exist_ok=True)

In [38]:
transform = transforms.Compose([ 
        transforms.Resize((224, 224)), transforms.ToTensor() 
    ])

In [39]:
## Loading the dataset for determination of the class counts
temp_dataset = BlindnessDataset(csv_file, pros_img_dir, transform=transform)
class_counts = temp_dataset.class_counts
max_count = class_counts.max()

In [40]:
dataset = BlindnessDataset(csv_file, pros_img_dir, transform=transform, max_count=max_count)
train_indices, val_indices = train_test_split(list(range(len(dataset))), test_size=0.2, stratify=dataset.annotations.iloc[:, 1])
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)
        

In [41]:
val_annotations = dataset.annotations.iloc[val_indices]
val_annotations.to_csv(val_csv_file, index=False)

for idx in val_indices:
    img_name = dataset.annotations.iloc[idx, 0] + '.png'
    src_path = os.path.join(pros_img_dir, img_name)
    dst_path = os.path.join(val_root_dir, img_name)
    shutil.copy2(src_path, dst_path)

In [42]:
def cross_validate_and_ensemble(train_loader, val_loader): 
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    models = {'efficientnet_b0': [], 'efficientnet_b1': [], 
              'efficientnet_b2': [], 'efficientnet_b3': []}
        
    for model_type in models.keys(): 
        model = train_model(model_type, train_loader, val_loader) 
        models[model_type].append(model) 
    return models

In [43]:
def get_predicted_labels(predictions):
    predicted_labels = np.argmax(predictions, axis=1)
    return predicted_labels

In [44]:
def load_ground_truth_labels(test_csv_file):
    test_df = pd.read_csv(test_csv_file)
    if test_df.shape[1] > 1:
        ground_truth_labels = test_df.iloc[:, 1].values
    else:
        ground_truth_labels = None
    return ground_truth_labels

In [45]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

In [46]:
def predict_ensemble(models, test_csv_file,test_root_dir): 
    transform = transforms.Compose([ transforms.Resize((224, 224)), transforms.ToTensor() ]) 
    test_dataset = BlindnessDataset(test_csv_file, test_root_dir, transform=transform, test=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
    predictions = [] 
    for model_type, model_list in models.items(): 
        for model in model_list: 
            model.eval() 
            model = model.to(device) 
            model_preds = [] 
            with torch.no_grad(): 
                for images in test_loader: 
                    images = images.to(device) 
                    outputs = model(images) 
                    preds = torch.softmax(outputs, dim=1) 
                    model_preds.append(preds.cpu().numpy()) 
            model_preds = np.concatenate(model_preds, axis=0)
            print(f"{model_type} model predictions shape: {model_preds.shape}")
            predictions.append(model_preds)
    
    print(f"Number of model predictions: {len(predictions)}")
    for i, pred in enumerate(predictions):
        print(f"Shape of predictions from model {i+1}: {pred.shape}")
    
    final_predictions = np.mean(predictions, axis=0)
    print(f"Final averaged predictions shape: {final_predictions.shape}")
    
    predicted_labels = get_predicted_labels(final_predictions)
    
    ground_truth_labels = load_ground_truth_labels(test_csv_file)
    
    if ground_truth_labels is not None:
        final_kappa = quadratic_weighted_kappa(ground_truth_labels, predicted_labels)
        print("Final Kappa:", final_kappa)
    else:
        final_kappa = None
        print("Ground truth labels are not available in the test CSV file.")
    
    return final_predictions, final_kappa

In [47]:
models = cross_validate_and_ensemble(train_loader, val_loader)

Epoch 1/25, Validation Loss: 3.425243536631266
Validation QWK: 0.10372731731240803
Epoch 2/25, Validation Loss: 3.02617857749002
Validation QWK: 0.20901126408010018
Epoch 3/25, Validation Loss: 2.773119598104243
Validation QWK: 0.2711651824909068
Epoch 4/25, Validation Loss: 2.5814951867388007
Validation QWK: 0.2971180821219077
Epoch 5/25, Validation Loss: 2.4160844401309363
Validation QWK: 0.3331246086412023
Epoch 6/25, Validation Loss: 2.2259604345288193
Validation QWK: 0.3806500761808025
Epoch 7/25, Validation Loss: 2.1242343752007735
Validation QWK: 0.41696383087111566
Epoch 8/25, Validation Loss: 1.905723854115135
Validation QWK: 0.4754076973255056
Epoch 9/25, Validation Loss: 1.9038859647617006
Validation QWK: 0.4579762057055137
Epoch 10/25, Validation Loss: 1.7319067444717675
Validation QWK: 0.5045472518782128
Epoch 11/25, Validation Loss: 1.696681088522861
Validation QWK: 0.5034366489430684
Epoch 12/25, Validation Loss: 1.6847244375630428
Validation QWK: 0.5126093669583118
Epoc

model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

Epoch 1/25, Validation Loss: 3.509630931051154
Validation QWK: 0.0328232971372161
Epoch 2/25, Validation Loss: 3.0375624393161975
Validation QWK: 0.08704504963094939
Epoch 3/25, Validation Loss: 2.7778269177988957
Validation QWK: 0.12812248186946007
Epoch 4/25, Validation Loss: 2.527450111874363
Validation QWK: 0.24071635501711885
Epoch 5/25, Validation Loss: 2.314150235109162
Validation QWK: 0.2728838256722744
Epoch 6/25, Validation Loss: 2.293821098511679
Validation QWK: 0.3707986017746706
Epoch 7/25, Validation Loss: 2.016940903245357
Validation QWK: 0.41683313357304563
Epoch 8/25, Validation Loss: 1.9282165054689373
Validation QWK: 0.45103127987979785
Epoch 9/25, Validation Loss: 1.785440420895292
Validation QWK: 0.49138287605128916
Epoch 10/25, Validation Loss: 1.7063665442299425
Validation QWK: 0.5195847750865052
Epoch 11/25, Validation Loss: 1.6952583319262455
Validation QWK: 0.5369660460021906
Epoch 12/25, Validation Loss: 1.655021259659215
Validation QWK: 0.5358082836958027
Ep

In [48]:
###DUE
final_predictions_val, final_kappa_val = predict_ensemble(models, val_csv_file,val_root_dir)

efficientnet_b0 model predictions shape: (1805, 5)
efficientnet_b1 model predictions shape: (1805, 5)
efficientnet_b2 model predictions shape: (1805, 5)
efficientnet_b3 model predictions shape: (1805, 5)
Number of model predictions: 4
Shape of predictions from model 1: (1805, 5)
Shape of predictions from model 2: (1805, 5)
Shape of predictions from model 3: (1805, 5)
Shape of predictions from model 4: (1805, 5)
Final averaged predictions shape: (1805, 5)
Final Kappa: 0.8510754897931223


In [ ]:
final_predictions, final_kappa = predict_ensemble(models, test_csv_file,test_root_dir)

In [ ]:
thresholds = [0.5, 1.5, 2.5, 3.5]
final_classes = np.digitize(predictions, bins=thresholds)

In [ ]:
save_predictions(final_classes)